In [ ]:
import numpy as np
import torch
import matplotlib.pyplot as plt

import topotorch.src.utils as _utils
import topotorch.src.geometry as _geom
import topotorch.src.mesher as _mesher
import topotorch.src.material as _mat
import topotorch.src.bc as _bc

import topotorch.src.fe_struct as _fea
import topotorch.src.mma as _mma
import topotorch.src.viz as _viz

_Disp = _fea.DisplacementField

In [ ]:
geom = _geom.BrepGeometry("topotorch/mesh/Lbeam.json")

mesh = _mesher.grid_mesh_brep(
  brep=geom,
  nelx_desired=80,
  nely_desired=80,
  dofs_per_node=2,
  gauss_order=3,
)

density = np.zeros(mesh.num_elems)
_viz.plot_grid_mesh(mesh, density)
_viz.plot_brep(geom)

In [ ]:
dens_filter = _utils.create_density_filter(
  coords=_utils.to_torch(mesh.elem_centers),
  cutoff_distance=0.05 * mesh.bounding_box.diag_length
)

In [ ]:
material_params = _mat.StructuralMaterial(
  youngs_modulus=1.0,
  poissons_ratio=0.30,
)

In [ ]:
hang_faces = _bc.identify_faces(mesh, edges=[geom.edges[2]])
n = len(hang_faces)
hang_face_val = [(_Disp.V, -5.0e-4 * np.ones(n))]


top_faces = _bc.identify_faces(mesh, edges=[geom.edges[5]])
n = len(top_faces)
xv = (_Disp.U, np.zeros(n))
yv = (_Disp.V, np.zeros(n))

top_face_val = [xv, yv]

fixed_bc = _bc.DirichletBC(elem_faces=top_faces, values=top_face_val, name="fix")
load_bc = _bc.NeumannBC(elem_faces=hang_faces, values=hang_face_val, name="load")

bc_list = [fixed_bc, load_bc]
bc = _bc.process_boundary_conditions(bc_list, mesh)

_viz.plot_bc(bc_list, mesh)

In [ ]:
force = torch.zeros((mesh.num_dofs,), dtype=torch.double)
force = force.index_add(
  0,
  _utils.to_torch(mesh.elem_dof_mat.flatten(), dtype=torch.long),
  bc["elem_forces"].flatten().double(),
)
bc["force"] = force

In [ ]:
fea = _fea.FEA(
  mesh=mesh,
  bc=bc,
  mat=material_params,
)

In [ ]:
def objective_function(density: torch.Tensor, fe: _fea.FEA, penal: float, beta):
  def objective_wrapper(density: torch.Tensor):
    # filter and project the density
    filt_dens = torch.matmul(dens_filter, density.view(-1))
    density = _utils.threshold_filter(filt_dens, beta=beta).view(-1)
    penal_dens = density**penal + 1e-2

    # material parameters
    young_0 = fe.mat.youngs_modulus
    youngs_mod = young_0 * penal_dens.view(-1)

    compliance, u = fea.loss_function(youngs_mod)
    compliance = compliance * 1e4
    return compliance, (u, penal_dens)
 
  compliance, (displacements, dens) = objective_wrapper(density)
  d_compliance = torch.autograd.grad(compliance, density)[0] 
  return compliance, d_compliance.reshape((-1, 1)), displacements, dens

In [ ]:
def constraint_function(density: torch.Tensor, max_vol_frac: float, beta):
  def constraint_wrapper(density: torch.Tensor):
    filt_dens = torch.matmul(dens_filter, density.view(-1))
    density = _utils.threshold_filter(filt_dens, beta=beta)

    vol_frac = torch.mean(density)

    print("Volume fraction:", vol_frac.item())
    return (vol_frac / max_vol_frac) - 1.0

  vol_cons = constraint_wrapper(density)
  d_vol_cons = torch.autograd.grad(vol_cons, density)[0]
  return vol_cons, d_vol_cons.T

In [ ]:
def optimize_design(
  fe: _fea.FEA,
  max_vol_frac: float,
  max_iter: int,
  move_limit: float = 1e-2,
  kkt_tol: float = 1e-5,
  step_tol: float = 1e-5,
  plot_interval: int = 5,
):

  design_var = max_vol_frac*np.ones((fe.mesh.num_elems, 1))
  num_design_var = design_var.shape[0]
  num_cons = 1
  lower_bound = np.zeros((num_design_var, 1))
  upper_bound = np.ones((num_design_var, 1))
  mma_params = _mma.MMAParams(
    max_iter=max_iter,
    kkt_tol=kkt_tol,
    step_tol=step_tol,
    move_limit=move_limit,
    num_design_var=num_design_var,
    num_cons=num_cons,
    lower_bound=lower_bound,
    upper_bound=upper_bound,
  )
  mma_state = _mma.init_mma(design_var, mma_params)

  while not mma_state.is_converged:
    print("mma_state.epoch", mma_state.epoch)
    penal = min(8.0, 1.0 + 0.05 * mma_state.epoch)
    beta = min(32.0, 1.0 + 0.1 * mma_state.epoch)

    density = torch.from_numpy(mma_state.x).requires_grad_(True).double()

    objective, grad_obj, u, dens = objective_function(density, fe, penal, beta)

    objective = objective.detach().numpy()
    grad_obj = grad_obj.detach().numpy()


    constr, grad_cons = constraint_function(density, max_vol_frac, beta)

    constr = constr.detach().numpy()
    grad_cons = grad_cons.detach().numpy()

    status = f"epoch {mma_state.epoch} J {objective.item():.2E} mc {constr.item():.2F}"
    print(status)

    if mma_state.epoch % plot_interval == 0:
      _, ax = plt.subplots(1, 1)
      node_deformation = torch.stack((u[0::2], u[1::2]), dim=1).detach().numpy()
      deformed_mesh = _mesher.deform_mesh(fe.mesh, node_deformation)
      ax = _viz.plot_grid_mesh(
        deformed_mesh,
        dens.detach().numpy().reshape(-1),
        ax=ax,
        # val_range=(0.0, 1.0),
        colorbar=True,
      )
      ax.set_aspect("equal")
      ax.spines[["top", "right", "left", "bottom"]].set_visible(False)
      ax.set_xticks([])
      ax.set_yticks([])
      plt.show()
      plt.pause(1e-6)

    mma_state = _mma.update_mma(
      mma_state, mma_params, objective, grad_obj, constr, grad_cons
    )

  return mma_state, u

In [ ]:
mma_state, u = optimize_design(fea, max_vol_frac=0.5, max_iter=250, move_limit=2.0e-2)